In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [2]:
import matplotlib.pyplot as plt
import os 

# Data Wrangling
import h5py
import numpy as np
import pandas as pd
from pathlib import Path
import glob
import math
import flammkuchen as fl

# Image
import cv2 

#eye detetcion
from skimage.segmentation import flood, flood_fill
from skimage.measure import label, regionprops, regionprops_table
import h5py
from tqdm import tqdm

#### Import custom functions

In [3]:
# my own utils
from utils_motato import compute_angle_between_vect_tail,compute_angle_between_vect,exptrapolate_segments

from eye_extraction_helpers import midpoint
from eye_extraction_helpers import mid_head
from eye_extraction_helpers import preprocess_img
from eye_extraction_helpers import angles, compute_angles

from feature_extractuion_helpers import nanzscore

#### Setup Params

In [4]:
# set up params that dont change 
fps_new = 200
fps = 200
N_seg = 10 


## Set up paths

In [5]:
master_path = Path(r'\\portulab.synology.me\data\Kata\testdata\Raw_Data')

fish_paths = list(master_path.glob('*f[0-9]*'))
fish_paths

[WindowsPath('//portulab.synology.me/data/Kata/testdata/Raw_Data/240423_f0')]

In [6]:
print ("{} videos found".format(len(fish_paths)))

1 videos found


In [7]:

for ind, fish_path in enumerate(tqdm(fish_paths)):
    print ('working on :', fish_path)
    my_file = Path(fish_path /"eye_coords.h5")
    if my_file.is_file():
        print ('file exists')
        pass

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]

working on : \\portulab.synology.me\data\Kata\testdata\Raw_Data\240423_f0
file exists


## loop over folder

In [8]:
### Check if eye_coords exist

for ind, fish_path in enumerate(tqdm(fish_paths)):
    print ('working on :', fish_path)
    my_file = Path(fish_path /"eye_coords.h5")
    if my_file.is_file():
        print ('file exists')
        pass

    vid_path = list(fish_path.glob('*video*'))[0]
    filename = list(fish_path.glob('*316000.h5*'))[0]
    df= pd.read_hdf(filename,  header=[1, 2], index_col=0)
    df = df['DLC_resnet50_dlc_2Dec12shuffle1_316000']
    print(f'{df.shape[0]/(fps*60)} minutes at {fps} fps')
    print('working on {} frames'.format(df.shape[0]))
    
    #Extract angles
    body_x = df.body.values[:, 0].astype('float')
    body_y = df.body.values[:, 1].astype('float')
    
    tail_x_col = [f'tail_{i}' for i in range(5)]
    tail_y_col = [f'tail_{i}' for i in range(5)]
    tail_x = np.array([df[x].iloc[:, 0].values.astype('float') for x in tail_x_col])
    tail_y = np.array([df[x].iloc[:, 1].values.astype('float') for x in tail_y_col])
    
    #Preprocess data 
    tail_x_10, tail_y_10 = exptrapolate_segments(tail_x, tail_y, 10)
    # computations
    mid_headx, mid_heady, left_mid_eye_x, left_mid_eye_y, right_mid_eye_x, right_mid_eye_y = mid_head(df)
    body_vect = np.vstack((mid_headx -body_x , mid_heady - body_y)) 

    # eye amgle loop
    frame_n = df.shape[0]
    eye_angles =[]
    eye_rot = []
    eye_verg = []
    eye_coords =[]

    cap = cv2.VideoCapture(str(vid_path))

    for i in range(frame_n):
        success = cap.grab()
        ret, image = cap.retrieve()
        mask_left = preprocess_img(image, left_mid_eye_y[i],left_mid_eye_x[i], tol_val=10)
        mask_right = preprocess_img(image, right_mid_eye_y[i],right_mid_eye_x[i], tol_val=10)
        left_eye_vect, right_eye_vect, l_eye_points, r_eye_points, [l_x0, l_y0], [r_x0, r_y0]= angles(i, mask_right, mask_left, body_x, body_y)
        left_eye_angle, right_eye_angle, rotation_eye, vergence = compute_angles(i, left_eye_vect, right_eye_vect, body_vect)
        eye_angles.append([left_eye_angle, right_eye_angle])
        eye_verg.append(vergence)
        eye_rot.append(rotation_eye)
        eye_coords.append([l_eye_points, r_eye_points])

    cap.release()

    #save data
    hf = h5py.File(fish_path/'eye_angles.h5', 'w')
    hf.create_dataset('eye_angles', data= np.asarray(eye_angles), compression="gzip", compression_opts=9)
    hf.close()

    hf = h5py.File(fish_path/'eye_verg.h5', 'w')
    hf.create_dataset('eye_verg', data= np.asarray(eye_verg), compression="gzip", compression_opts=9)
    hf.close()

    hf = h5py.File(fish_path/'eye_rot.h5', 'w')
    hf.create_dataset('eye_rot', data= np.asarray(eye_rot), compression="gzip", compression_opts=9)
    hf.close()

    hf = h5py.File(fish_path/'eye_coords.h5', 'w')
    hf.create_dataset('eye_coords', data= np.asarray(eye_coords), compression="gzip", compression_opts=9)
    hf.close()




  0%|                                                                                            | 0/1 [00:00<?, ?it/s]

working on : \\portulab.synology.me\data\Kata\testdata\Raw_Data\240423_f0
file exists
7.9519166666666665 minutes at 200 fps
working on 95423 frames


  0%|                                                                                            | 0/1 [00:05<?, ?it/s]


KeyboardInterrupt: 